<a href="https://colab.research.google.com/github/JJxx5201/jayjay/blob/main/%E6%9C%83%E8%AD%B0%E8%A8%98%E9%8C%84%E5%B0%8F%E5%B7%A5%E5%85%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests google-generativeai assemblyai # assemblyai SDK 名稱可能不同

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.5 MB/s eta 0:00:00


In [7]:
import os
from google.colab import userdata # 建議使用 Secrets
import requests
import time
import google.generativeai as genai

# 假設使用者已在 Colab Secrets 中設定 'ASSEMBLYAI_API_KEY' 和 'GOOGLE_API_KEY'
ASSEMBLYAI_API_KEY = userdata.get('ASSEMBLYAI_API_KEY')
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

if not ASSEMBLYAI_API_KEY or not GEMINI_API_KEY:
  print("⚠️ 請先在 Colab Secrets (左側鑰匙圖示) 中設定 AssemblyAI 和 Gemini API Keys。")
else:
  print("✅ API Keys 載入成功。")
  # 設定 Gemini SDK
  try:
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Gemini SDK 設定成功。")
  except Exception as e:
    print(f"⚠️ Gemini SDK 設定失敗: {e}")

# AssemblyAI API 端點
assemblyai_upload_endpoint = "https://api.assemblyai.com/v2/upload"
assemblyai_transcript_endpoint = "https://api.assemblyai.com/v2/transcript"

✅ API Keys 載入成功。
✅ Gemini SDK 設定成功。


In [8]:
from google.colab import files
import os

print("請上傳您的音訊檔案：")
uploaded = files.upload()

if not uploaded:
  print("\n沒有上傳檔案。")
  audio_file_path = None
else:
  # 取得第一個上傳檔案的名稱和路徑
  audio_file_name = list(uploaded.keys())[0]
  audio_file_path = os.path.join(os.getcwd(), audio_file_name)
  print(f"\n📄 檔案 '{audio_file_name}' 已上傳至 '{audio_file_path}'")

請上傳您的音訊檔案：


Saving 錄製 (6).m4a to 錄製 (6).m4a

📄 檔案 '錄製 (6).m4a' 已上傳至 '/content/錄製 (6).m4a'


In [9]:
def upload_audio_to_assemblyai(api_key, file_path):
    headers = {'authorization': api_key}
    try:
        with open(file_path, 'rb') as f:
            response = requests.post(assemblyai_upload_endpoint, headers=headers, data=f)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()['upload_url']
    except requests.exceptions.RequestException as e:
        print(f"❌ 上傳音訊時發生錯誤: {e}")
        if response:
            print(f"   回應內容: {response.text}")
        return None
    except FileNotFoundError:
        print(f"❌ 找不到檔案: {file_path}")
        return None

def start_transcription(api_key, audio_url):
    json_data = {
        "audio_url": audio_url,
        "language_code": "zh", # 指定中文
        "speaker_labels": True
    }
    headers = {
        "authorization": api_key,
        "content-type": "application/json"
    }
    try:
        response = requests.post(assemblyai_transcript_endpoint, json=json_data, headers=headers)
        response.raise_for_status()
        return response.json()['id']
    except requests.exceptions.RequestException as e:
        print(f"❌ 開始轉錄時發生錯誤: {e}")
        if response:
             print(f"   回應內容: {response.text}")
        return None

def poll_transcript_result(api_key, transcript_id):
    polling_endpoint = f"{assemblyai_transcript_endpoint}/{transcript_id}"
    headers = {"authorization": api_key}
    while True:
        try:
            response = requests.get(polling_endpoint, headers=headers)
            response.raise_for_status()
            result = response.json()
            status = result['status']
            print(f"   轉錄狀態: {status}")
            if status == 'completed':
                return result # 回傳完整的結果 JSON
            elif status == 'error':
                print(f"❌ 轉錄失敗: {result.get('error', '未知錯誤')}")
                return None
            elif status in ['queued', 'processing']:
                time.sleep(5) # 等待 5 秒再查詢
            else:
                print(f"   收到未預期的狀態: {status}")
                time.sleep(5)
        except requests.exceptions.RequestException as e:
             print(f"❌ 查詢轉錄結果時發生錯誤: {e}")
             if response:
                 print(f"   回應內容: {response.text}")
             time.sleep(5) # 發生錯誤時也稍等一下再重試
        except Exception as e:
             print(f"❌ 處理輪詢時發生未知錯誤: {e}")
             return None # 發生未知錯誤，停止輪詢

def format_transcript_python(words_list):
    if not words_list:
        return "", []

    formatted_text = ""
    current_speaker = None
    current_line = ""
    start_time_str = ""
    speakers = set()

    for word in words_list:
        speaker_label = f"講者{word['speaker']}"
        speakers.add(speaker_label)
        timestamp = word['start'] / 1000 # 轉為秒
        minutes = int(timestamp // 60)
        seconds = int(timestamp % 60)
        current_time_str = f"{minutes:02d}:{seconds:02d}"

        if current_speaker != speaker_label: # 換講者或第一句
            if current_line: # 先結束上一行
                formatted_text += f"{start_time_str} {current_speaker}: {current_line.strip()}\n"
            # 開始新的一行
            current_speaker = speaker_label
            start_time_str = current_time_str
            current_line = word['text']
        else: # 同一個講者，繼續加字
            current_line += f" {word['text']}"

    # 加入最後一行
    if current_line:
         formatted_text += f"{start_time_str} {current_speaker}: {current_line.strip()}\n"

    speaker_mapping_lines = [f"【講者對應】"]
    for sp in sorted(list(speakers)):
         speaker_mapping_lines.append(f"{sp} 代表 ______")

    return "\n".join(speaker_mapping_lines) + "\n\n【逐字稿內容】\n" + formatted_text.strip(), sorted(list(speakers))


# --- 執行轉錄 ---
transcript_text = None
transcript_json = None
if audio_file_path and ASSEMBLYAI_API_KEY:
    print("\n⬆️ 開始上傳音訊至 AssemblyAI...")
    upload_url = upload_audio_to_assemblyai(ASSEMBLYAI_API_KEY, audio_file_path)

    if upload_url:
        print(f"   上傳完成，URL: {upload_url}")
        print("\n⏳ 開始請求轉錄...")
        transcript_id = start_transcription(ASSEMBLYAI_API_KEY, upload_url)

        if transcript_id:
            print(f"   轉錄請求成功，ID: {transcript_id}")
            print("\n🔄 開始輪詢轉錄結果 (可能需要幾分鐘)...")
            transcript_json = poll_transcript_result(ASSEMBLYAI_API_KEY, transcript_id)

            if transcript_json and transcript_json.get('status') == 'completed':
                print("\n✅ 轉錄完成！")
                # 格式化逐字稿
                transcript_text, speaker_list = format_transcript_python(transcript_json.get('words'))
                print("\n--- 逐字稿 ---")
                print(transcript_text)
                # 您也可以選擇將 transcript_json (原始 JSON) 儲存起來
                # import json
                # print("\n--- 原始 JSON (部分) ---")
                # print(json.dumps(transcript_json, indent=2, ensure_ascii=False)[:1000] + "...") # 只印前1000字元
            else:
                print("\n❌ 無法完成轉錄。")
        else:
             print("\n❌ 無法啟動轉錄請求。")
    else:
         print("\n❌ 音訊上傳失敗。")
elif not audio_file_path:
    print("\n請先上傳音訊檔案。")
else:
    print("\n請先設定 AssemblyAI API Key。")


⬆️ 開始上傳音訊至 AssemblyAI...
   上傳完成，URL: https://cdn.assemblyai.com/upload/d656650c-7b55-4434-a20f-2feb0d017a72

⏳ 開始請求轉錄...
   轉錄請求成功，ID: 0a419bdf-06c3-4231-b4f7-b67e4663246e

🔄 開始輪詢轉錄結果 (可能需要幾分鐘)...
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: processing
   轉錄狀態: completed

✅ 轉錄完成！

--- 逐字稿 ---
【講者對應】
講者A 代表 ______
講者B 代表 ______
講者C 代表 ______

【逐字稿內容】
00:01 講者A: 這也 是 想要 跟 你 們 做 的,應該 說 先確 認左側 這個 部分 的 牌序 會 是 什麼 吧? 左側? 好, 我會問 一些 你們 剛在 那個 家裡 的 方便 的 牌序 第一 個 是 原來 research 其實 就 有 這個 建議 關鍵 字 跟 歷史 的 搜尋詞 相關 的 一些 內容 那 如果 加上 這個 下次 選單 可以 用到 換點 嗎? 需要 花一工 去 把 它 灌 掉 不然 兩者會 錯誤 那現 在 R5 怎麼 做 直接 蓋著現 在 我 看 他們 

In [ ]:
# Cell 5: Gemini Summarization (using Gemini 1.5 Pro)

# 確保導入必要的函式庫
try:
    import google.generativeai as genai
    from IPython.display import display, Markdown
    import time # 雖然主要在 Cell 4 用，但以防萬一導入
except ImportError:
    print("正在安裝 google-generativeai...")
    !pip install google-generativeai -q
    import google.generativeai as genai
    from IPython.display import display, Markdown
    print("安裝完成，請重新執行此 Cell。")
    # 如果需要，可以 raise Exception("請重新執行此 Cell") 來停止執行

def generate_gemini_summary(api_key, transcript, model_name="gemini-1.5-pro-latest"): # 將預設值改為 1.5 Pro
    """使用指定的 Gemini 模型產生會議摘要"""
    if not transcript:
        print("❌ 沒有逐字稿內容，無法產生摘要。")
        return None
    if not api_key:
        print("❌ 請先設定 Gemini API Key (在 Cell 2)。")
        return None

    # --- 開始 try 區塊 ---
    try:
        # 確保 Gemini SDK 已配置
        # 檢查 genai._client 是否存在且有效可能更健壯，但這裡先簡化
        # 如果之前的 Cell 已經成功配置過，這裡可能不需要重複配置
        try:
           # 檢查配置狀態，如果未配置或API Key不同則重新配置
           current_conf = genai.get_config()
           if not current_conf or current_conf.get('api_key') != api_key:
              print("   嘗試設定/更新 Gemini SDK 配置...")
              genai.configure(api_key=api_key)
              print("   Gemini SDK 配置完成。")
        except Exception as config_err:
           print(f"   配置 Gemini SDK 時遇到問題: {config_err}. 嘗試強制配置...")
           genai.configure(api_key=api_key)
           print("   Gemini SDK 強制配置完成。")


        print(f"   選用模型: {model_name}")
        model = genai.GenerativeModel(model_name)

        # 準備 Prompt (與之前相同)
        prompt = f"""
{transcript}
-----------------------------
我希望將上面的逐字稿彙整成以下項目:

- 摘要 (150字內)
- TODOs（標題） + 負責人 + 項目的細節描述（詳細描述避免缺失、把同一人負責的項目 group 在一起）
- 重點整理方法:
  - 如果是「產品會議」依照「每一項產品為標題」展出「重點與決議」「細節描述」跟「次要事項＆描述」（詳細描述避免缺失）
  - 如果是「行銷會議」依照「通路、或是社群媒體、或是品牌」為標題，展出「重點與決議」「細節描述」跟「次要事項＆描述」（詳細描述避免缺失）
  - 如果是「廣泛議題的會議」依照「主題、議題」為標題，展出「重點與決議」「細節描述」跟「次要事項＆描述」（詳細描述避免缺失）
  - 如果是「跨部門會議」依照「部門別」為標題，展出「重點與決議」「細節描述」跟「次要事項＆描述」（詳細描述避免缺失）
  - 都要:無結論的內容（標題） + 細節描述（詳細描述避免缺失）
  - 都要:矛盾或奇怪內容（標題） + 細節描述（詳細描述避免缺失）
- 特別注意要求:
  - 以上的議題整理，希望細節重點都加上逐字稿時間格式為 MM:SS
  - 以上請盡可能詳細避免漏掉細節
  - 使用 markdown 語法 ( 外圍不要使用 code block 包起來)
  - 講者對應不要忘記了，但輸出時不要再出現講者對應
  - 直接開始不要有，好的之類的詞，這個輸出最終要輸出成 PDF 的
"""
        print(f"\n🤖 正在向 Gemini ({model_name}) 請求摘要...")
        # 注意：1.5 Pro 模型處理長文本可能比 Flash 模型稍慢
        response = model.generate_content(prompt)
        print(f"✅ 收到 Gemini ({model_name}) 回應。")

        # 檢查回應中是否有 text 屬性
        if hasattr(response, 'text'):
            return response.text
        else:
             # 如果沒有 text 屬性，可能需要檢查 response.parts
             if hasattr(response, 'parts') and response.parts:
                 # 假設第一個 part 是所需的文本
                 return "".join(part.text for part in response.parts) # 合併所有 parts 的 text
             else:
                 # 如果結構未知，打印原始回應以供調試
                 print(f"⚠️ Gemini ({model_name}) 回應結構未知或無內容: {response}")
                 # 檢查是否有停止原因 (e.g., 安全過濾)
                 if hasattr(response, 'prompt_feedback'):
                     print(f"   提示回饋: {response.prompt_feedback}")
                 return None

    # --- 這裡是 except 區塊 ---
    except Exception as e:
        import traceback # 導入 traceback 來印出更詳細的錯誤
        print(f"❌ 請求 Gemini ({model_name}) 摘要時發生錯誤: {e}")
        print(traceback.format_exc()) # 印出完整的錯誤追蹤
        # 嘗試印出更詳細的 API 錯誤 (如果有的話)
        if hasattr(e, 'response') and e.response:
            print(f"   API 回應錯誤: {e.response.text}")
        return None

# --- 執行摘要 ---
summary_text_pro = None # 使用新變數名以示區別
# 確保 transcript_text 變數存在且有內容 (來自 Cell 4 的結果)
# 並且確保 GEMINI_API_KEY 存在 (來自 Cell 2)
if 'transcript_text' in locals() and transcript_text and 'GEMINI_API_KEY' in locals() and GEMINI_API_KEY:
    # 明確指定使用 gemini-1.5-pro-latest 模型
    model_to_use = "gemini-1.5-pro-latest"
    summary_text_pro = generate_gemini_summary(GEMINI_API_KEY, transcript_text, model_name=model_to_use)

    if summary_text_pro:
        print(f"\n--- 會議摘要 (使用 {model_to_use}) ---")
        # 使用 IPython.display.Markdown 來良好地顯示 Markdown 格式
        display(Markdown(summary_text_pro))
    else:
        print(f"\n❌ 無法使用 {model_to_use} 產生摘要。")
elif not ('transcript_text' in locals() and transcript_text):
    print("\n沒有有效的逐字稿可供摘要 (請確認 Cell 4 已成功執行)。")
elif not ('GEMINI_API_KEY' in locals() and GEMINI_API_KEY):
     print("\n請先在 Cell 2 中成功載入 Gemini API Key。")
else:
    print("\n發生未知問題，無法執行摘要。")

   配置 Gemini SDK 時遇到問題: module 'google.generativeai' has no attribute 'get_config'. 嘗試強制配置...
   Gemini SDK 強制配置完成。
   選用模型: gemini-1.5-pro-latest

🤖 正在向 Gemini (gemini-1.5-pro-latest) 請求摘要...
✅ 收到 Gemini (gemini-1.5-pro-latest) 回應。

--- 會議摘要 (使用 gemini-1.5-pro-latest) ---


## 摘要

會議討論了多項產品功能更新、行銷策略以及跨部門合作議題。其中，重點包括 APP 八項效率化工具預計 7/3 完成 (00:00)，廣告帳號初始化功能開發，區分電商及非電商架構 (01:05)，LINE 串接問題待修復及費用協商 (03:49)，測試貝多納及其他商店的 LINE 投放成效 (05:44)，以及電通需求的報表篩選器新增 (11:31)。此外，也提及導流歸因報表加入 UTM 參數及自動化標記功能 (15:36)，以及保養抗式 meta 與 AI 社群廣告版位規劃 (19:55)。會議中亦介紹了新進成員及各部門負責人。


## TODOs

**講者 A:**

* 與 Henry 討論在婚姻報表中加入 UDN 維度需求 (13:56)。細節：UDN 收到的資訊直接在小球規呈現，方便投手判斷，並防止外部人士竄改商店數據 (14:08)。需確認是否使用 Pay2Pay 比對，以及自動標記 UTM 參數的可行性 (16:03)。
* 與講者 D 和運喬開會討論 AI 社群廣告版位規劃 (22:00)。
* 向講者 F 學習 GF4 課程 (22:43)。

**講者 C:**

* 與 LINE 協商測試環境投放費用吸收 (06:59)。
* 與 AD2 開會討論 LINE 廣告後台看不到曝光的問題 (07:43)，時間：下午四點。
* 與 Jiggy 和 Bruce 討論其他商店 LINE 投放測試，不指定受眾狀況下的成效 (05:44)。
* 與講者 D 確認 secondary 和 miniclip 商店測試的可行性 (06:03)。與講者 D 排時間討論 secondary 商店的 LAP 廣告測試，因為其並非點擊導流而是導人流至代售頁面 (06:26)。
* 向講者 D 解釋在華盛頓幾家小額測試使用的是客戶的預算 (07:04)。

**講者 D:**

* 與講者 C 確認 secondary 和 miniclip 商店測試的可行性 (06:03)。與講者 C 排時間討論 secondary 商店的 LAP 廣告測試，並非點擊導流而是導人流至代售頁面 (06:26)。
* 與 K 哥確認 AI 社群廣告版位的進度 (20:55)。
* 與運喬和講者 A 開會更新 AI 社群廣告版位規劃進度 (22:07)。

**講者 F:**

* 確認 CPS 恆端 RAW 測試，預計禮拜三上線 (03:33)。
* 與兩位 Boris 討論 CPS 恆端 RAW 設定 (03:33)。
* 與講者 C 討論貝多納的成效 (06:35)。


## 產品會議 - APP

**重點與決議:** APP 八項效率化工具預計 7/3 完成 (00:00)，廣告帳號初始化功能開發，區分電商及非電商架構 (01:05)

**細節描述:** 八項效率化工具旨在縮減投手建廣告的時間 (00:00)，初始化功能可直接建立對應架構，例如電商架構的新客舊客分類，共十二個基本架構 (00:00)。

**次要事項＆描述:** RAWENGINE 將與廣告優化結合，統一提升投手廣告操作效率 (00:00)。目前除了上表工具外，還有廣告帳號初始化功能 (00:00)。


## 跨部門會議 - 與 LINE 合作

**重點與決議:** LINE 串接問題待修復及費用協商 (03:49)，測試貝多納及其他商店的 LINE 投放成效 (05:44)

**細節描述:**  LINE 串接存在問題，LINE 內部追不到兔回傳 LAP 的數據，AD2 則在指定大量體時無法曝光，兩者皆需修復 (04:51)。已與 LINE 協商第一次延期，將再次協商第二次延期時間，待產品串接確認後再議 (04:51)。貝多納持續測試，預算控制在 20 以內 (05:44)。其他商店可先測試不指定受眾的投放成效，再測試指定受眾的成效 (05:44)。北港村將協助爭取 LINE 測試環境投放的費用吸收 (06:59)。

**次要事項＆描述:** LINE Today 版位已串進 91AD 平台，但廣告後台看不到曝光，LINE 卻表示有曝光 (07:41)。AD2 使用 GAM 串接 91AD，掛 91AD 的名字 (07:58)。下午四點將與 AD2 開會討論此問題 (07:43)。


## 行銷會議 - 電通需求

**重點與決議:** 電通需求的報表篩選器新增 (11:31)

**細節描述:** 電通在報表分析時，需要在某個 Explorer 中增加年齡篩選器，以濾除不合理的會員年齡數據，提高分析的有效性 (11:31)。目前 91APP 會員年齡由會員自行填寫，但存在亂填的情況 (11:31)。


## 無結論的內容

**細節描述:** 討論 APP 內部部門分工及成員介紹 (01:05 - 02:50)，討論 Email 功能的延期 (09:54)，討論 Lightroom 的測試完成 (09:54)，UPD Slack 討論 Email 量能卡關問題 (09:54)，討論 LiveToday AD2 只支援產品目錄形式，需開發新的流量廣告格式 (09:40)，討論 meta 與 google API 上稿工具 (19:32)。


## 矛盾或奇怪內容

**細節描述:** 講者 A 表示林東北不應用 OK 來排需求，核心是九月份商店都在看導流規定 (14:54)。講者 A 提到希望 UTM 參數自動化標記，方便投手操作，但講者 F 表示 Google 沒有自動參數機制 (16:27 - 18:17)。講者 B 提及 980P 已串接 Meta 和 Google API 來上稿，但講者 F 表示 Google 還沒 (19:32 - 19:42)。

In [6]:
# --- Cell 2: 儲存結果至 Google Drive ---

# 確保導入必要的函式庫
import os
from google.colab import drive
from datetime import datetime

# 檢查 'transcript_text' 和 'summary_text' 變數是否存在
# 這兩個變數應由前一個 Cell 產生
if 'transcript_text' not in locals() and 'summary_text' not in locals():
    print("🤷‍ 找不到逐字稿或摘要內容。請先執行產生摘要的 Cell。")
else:
    print("\n--- 準備儲存檔案至 Google Drive ---")
    try:
        # 掛接 Google Drive
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive 已成功掛接！")

        # --- 請確認您的儲存路徑 ---
        GDRIVE_FOLDER_PATH = '/content/drive/MyDrive/會議記錄'

        # 檢查並建立目標資料夾
        if not os.path.exists(GDRIVE_FOLDER_PATH):
            os.makedirs(GDRIVE_FOLDER_PATH)
            print(f"📁 已成功建立資料夾: {GDRIVE_FOLDER_PATH}")

        # 產生時間戳記，避免檔案覆蓋
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        # 儲存逐字稿 (如果存在)
        if 'transcript_text' in locals() and transcript_text:
            transcript_file_name = f"meeting_transcript_{timestamp}.txt"
            full_transcript_path = os.path.join(GDRIVE_FOLDER_PATH, transcript_file_name)
            with open(full_transcript_path, "w", encoding="utf-8") as f:
                f.write(transcript_text)
            print(f"💾 逐字稿已儲存至: {full_transcript_path}")

        # 儲存摘要 (如果存在)
        if 'summary_text' in locals() and summary_text:
            summary_file_name = f"meeting_summary_{timestamp}.md"
            full_summary_path = os.path.join(GDRIVE_FOLDER_PATH, summary_file_name)
            with open(full_summary_path, "w", encoding="utf-8") as f:
                f.write(summary_text)
            print(f"💾 摘要已儲存至: {full_summary_path}")

        print("\n✨ 所有檔案已成功儲存到您的 Google Drive！")

    except Exception as e:
        print(f"❌ 存檔至 Google Drive 過程發生錯誤: {e}")



--- 準備儲存檔案至 Google Drive ---
Mounted at /content/drive
✅ Google Drive 已成功掛接！
💾 逐字稿已儲存至: /content/drive/MyDrive/會議記錄/meeting_transcript_2025-06-20_03-46-01.txt

✨ 所有檔案已成功儲存到您的 Google Drive！
